In [0]:
service_credential = dbutils.secrets.get(scope="blockpulse_keyvault_scope",key="blockpulsedatabrickssecret")

spark.conf.set("fs.azure.account.auth.type.blockpulsecrypto.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.blockpulsecrypto.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.blockpulsecrypto.dfs.core.windows.net", "39057514-1421-4818-9f98-8bcc69c384d4")
spark.conf.set("fs.azure.account.oauth2.client.secret.blockpulsecrypto.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.blockpulsecrypto.dfs.core.windows.net", "https://login.microsoftonline.com/0dba45aa-3354-4d4c-8c45-eea7b490aebe/oauth2/token")

In [0]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import *
from delta.tables import *
from datetime import datetime

spark= SparkSession.builder.appName("BlockPulse Bronze to Silver Pipeline").getOrCreate()

#Move Data from broze to Silver Layer
storage_account='blockpulsecrypto'
container='blockpulsedatacontainer'
today=datetime.now()
current_year=today.year
current_month=today.month
current_day=today.day

bronze_data= f"abfss://{container}@{storage_account}.dfs.core.windows.net/bronze/year={current_year}/month={current_month:02d}/day={current_day:02d}/data.json"
silver_data= f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/year={current_year}/month={current_month:02d}/day={current_day:02d}/"

bronze_df=spark.read.format("json").load(bronze_data)

print(f"This is the length of all the data in the bronze folder: {bronze_df.count()}")

display(bronze_df.show(5))

This is the length of all the data in the bronze folder: 100
+--------+---------------------+--------------------+----------+---------------------+--------------------+--------------------+-------------+-----------------------+--------+-----------+--------------------+--------------------+--------+-------------+---------------------+--------------------------------+---------------+----------+--------+--------------------+---------------------------+--------------------+------+--------------------+---------------+
|     ath|ath_change_percentage|            ath_date|       atl|atl_change_percentage|            atl_date|  circulating_supply|current_price|fully_diluted_valuation|high_24h|         id|               image|        last_updated| low_24h|   market_cap|market_cap_change_24h|market_cap_change_percentage_24h|market_cap_rank|max_supply|    name|    price_change_24h|price_change_percentage_24h|                 roi|symbol|        total_supply|   total_volume|
+--------+-------------

In [0]:

base_fact_df = bronze_df.select(
    col("id").alias("crypto_id"), 
    to_date(col("last_updated")).alias("snapshot_date"),   # extract date from timestamp
    col("current_price").alias("current_price_usd"),
    col("market_cap").alias("market_cap_usd"),
    col("total_volume").alias("total_volume_usd"),
    col("price_change_24h").alias("price_change_24h_usd"),
    col("price_change_percentage_24h").alias("price_change_pct_24h"),
    col("market_cap_rank").alias("market_cap_rank"),
    col("circulating_supply").alias("circulating_supply"),
    col("total_supply").alias("total_supply"),
    col("max_supply").alias("max_supply"),
    col("ath").alias("all_time_high"),
    col("ath_change_percentage").alias("ath_change_pct"),
    col("atl").alias("all_time_low"),
    col("atl_change_percentage").alias("atl_change_pct"),
    col("high_24h").alias("high_24h_usd"),
    col("low_24h").alias("low_24h_usd")
)

window_spec = Window.partitionBy("crypto_id").orderBy("snapshot_date")

crypto_market_snapshot_fact_df = base_fact_df.withColumn(
    "prev_volume_usd",
    lag("total_volume_usd").over(window_spec)
).withColumn(
    "volume_change_pct_24h",
    round(
        ((col("total_volume_usd") - col("prev_volume_usd")) / col("prev_volume_usd")) * 100, 2
    )
)

crypto_asset_dim_df = bronze_df.select(
    col("id").alias("crypto_id"),
    "symbol",
    "name",
    "image"
).dropDuplicates(["crypto_id"])

date_dim = bronze_df.select(
    to_date("last_updated").alias("date")
).dropDuplicates(["date"])

date_dim_df = date_dim \
    .withColumn("year", year("date").cast(LongType())) \
    .withColumn("month", month("date").cast(LongType())) \
    .withColumn("day", dayofmonth("date").cast(LongType())) \
    .withColumn("day_of_week", dayofweek("date").cast(LongType()))



# Save the crypto_market_snapshot_fact DataFrame for today in Parquet Format
crypto_market_snapshot_fact_df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save(silver_data+"crypto_market_snapshot_fact")

# Save the crypto_asset_dim DataFrame for today in Parquet Format
crypto_asset_dim_df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save(silver_data+"crypto_asset_dim")


# Save the date_dim DataFrame for today in Parquet Format
date_dim_df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save(silver_data+"date_dim")


date_dim_df.show(5)

crypto_asset_dim_df.show(5)

crypto_market_snapshot_fact_df.show(5)



+----------+----+-----+---+-----------+
|      date|year|month|day|day_of_week|
+----------+----+-----+---+-----------+
|2025-07-27|2025|    7| 27|          1|
+----------+----+-----+---+-----------+

+-----------+--------+--------------------+--------------------+
|  crypto_id|  symbol|                name|               image|
+-----------+--------+--------------------+--------------------+
|   fartcoin|fartcoin|            Fartcoin|https://coin-imag...|
|   algorand|    algo|            Algorand|https://coin-imag...|
|hyperliquid|    hype|         Hyperliquid|https://coin-imag...|
|   fetch-ai|     fet|Artificial Superi...|https://coin-imag...|
|    story-2|      ip|               Story|https://coin-imag...|
+-----------+--------+--------------------+--------------------+
only showing top 5 rows
+-----------+-------------+-----------------+--------------+----------------+--------------------+--------------------+---------------+--------------------+-------------------+----------+---